In [1]:
"""Parses the mmCIF file format."""
import collections
import dataclasses
import functools
import io
from typing import Any, Mapping, Optional, Sequence, Tuple
from absl import logging
import numpy as np
import os

from Bio import PDB
from Bio.Data import SCOPData
from Bio.Data.IUPACData import protein_letters_3to1

import se3_diffuse.mmcif_parsing
from se3_diffuse.process_pdb_dataset import process_mmcif, _retrieve_mmcif_files, process_serially
from se3_diffuse import errors
import time

In [2]:


from tqdm import tqdm

In [ ]:
#run from command line on alternate repo for use
#run in top level se3_diffusion directory
python process_pdb_dataset.py --mmcif_dir /mnt/h/datasets/mmCIF/ --max_resolution 3.0 --max_len 200 --write_dir /mnt/h/datasets/p200 --num_processes 22 --verbose


In [4]:
import gzip
import shutil


def process_serially2(
        all_mmcif_paths, max_resolution, max_len, write_dir, simple=False):
    all_metadata = []
    failed_paths = []
    for i, mmcif_path in enumerate(all_mmcif_paths):
        try:
            start_time = time.time()
            metadata = process_mmcif(
                mmcif_path,
                max_resolution,
                max_len,
                write_dir, simple=simple)
            elapsed_time = time.time() - start_time
            print(f'Finished {mmcif_path} in {elapsed_time:2.2f}s')
            all_metadata.append(metadata)
        except Exception as e:
            print(f'Failed {mmcif_path}: {e}')
            failed_paths.append(mmcif_path)
    return all_metadata, failed_paths


def gunzip(file_path,output_path):
    with gzip.open(file_path,"rb") as f_in, open(output_path,"wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
        f_in.close()
        f_out.close()
    os.remove(file_path)

# f='TEST.txt.gz'
# gunzip(f,f.replace(".gz",""))

In [8]:
mmcif_dir = '/mnt/h/datasets/mmCIF/'

max_file_size = 300000000
min_file_size = 100

max_res = 3.0
max_length = 128
output_dir = '/mnt/h/datasets/processed_128L'

all_mmcif_paths = _retrieve_mmcif_files(mmcif_dir, max_file_size, min_file_size, False)
total_num_files =0
for mmcif_file in os.listdir(mmcif_dir):
    if not mmcif_file.endswith('.cif'):
        continue
    mmcif_path = os.path.join(mmcif_dir, mmcif_file)
    total_num_files += 1
    if min_file_size <= os.path.getsize(mmcif_path) <= max_file_size:
        all_mmcif_paths.append(mmcif_path)


all_metadata, failed_paths= process_serially(all_mmcif_paths,max_res,max_length,
                                              output_dir, simple=False)

Gathering mmCIF paths


100%|███████████████████████████████████████████████████████████████████████████████| 1060/1060 [00:27<00:00, 38.30it/s]


Processing 21133 files our of 21136
Failed /mnt/h/datasets/mmCIF/00/200l.cif: Too long 272
Failed /mnt/h/datasets/mmCIF/01/101m.cif: Too long 295
Failed /mnt/h/datasets/mmCIF/02/102l.cif: Too long 299
Failed /mnt/h/datasets/mmCIF/03/103l.cif: Too long 290
Failed /mnt/h/datasets/mmCIF/03/103m.cif: Too long 272
Failed /mnt/h/datasets/mmCIF/05/105m.cif: Too long 271
Failed /mnt/h/datasets/mmCIF/05/205l.cif: Too long 277
Failed /mnt/h/datasets/mmCIF/06/106m.cif: Too long 300
Failed /mnt/h/datasets/mmCIF/07/107m.cif: Too long 292
Failed /mnt/h/datasets/mmCIF/07/207l.cif: Too long 254
Failed /mnt/h/datasets/mmCIF/08/108m.cif: Too long 243
Failed /mnt/h/datasets/mmCIF/08/208l.cif: Too long 171
Failed /mnt/h/datasets/mmCIF/09/209l.cif: Too long 186
Failed /mnt/h/datasets/mmCIF/10/210l.cif: Too long 273
Failed /mnt/h/datasets/mmCIF/11/211l.cif: Too long 298
Failed /mnt/h/datasets/mmCIF/12/112m.cif: Too long 294
Failed /mnt/h/datasets/mmCIF/13/213l.cif: Too long 238
Failed /mnt/h/datasets/mmCIF/

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 242.
  warnings.warn(
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 266.
  warnings.warn(


Failed /mnt/h/datasets/mmCIF/a0/2a0j.cif: Too long 192
Failed /mnt/h/datasets/mmCIF/a0/3a02.cif: Too long 152
Failed /mnt/h/datasets/mmCIF/a0/3a03.cif: Too long 162
Failed /mnt/h/datasets/mmCIF/a0/3a0d.cif: Too long 229
Failed /mnt/h/datasets/mmCIF/a0/3a0e.cif: Too long 253
Failed /mnt/h/datasets/mmCIF/a0/3a0s.cif: Too long 152
Failed /mnt/h/datasets/mmCIF/a0/3a0t.cif: Too long 262
Failed /mnt/h/datasets/mmCIF/a0/3a0u.cif: Too long 213
Failed /mnt/h/datasets/mmCIF/a0/3a0v.cif: Too long 210
Failed /mnt/h/datasets/mmCIF/a0/3a0x.cif: Too long 245
Failed /mnt/h/datasets/mmCIF/a0/4a0i.cif: Too long 283
Failed /mnt/h/datasets/mmCIF/a0/4a0j.cif: Too long 294
Failed /mnt/h/datasets/mmCIF/a0/4a0n.cif: Too long 136
Failed /mnt/h/datasets/mmCIF/a0/4a0x.cif: Too long 212
Failed /mnt/h/datasets/mmCIF/a0/5a0b.cif: Too long 293
Failed /mnt/h/datasets/mmCIF/a0/5a0f.cif: Too long 261
Failed /mnt/h/datasets/mmCIF/a0/5a0o.cif: Too long 234
Failed /mnt/h/datasets/mmCIF/a0/6a0a.cif: Too long 233
Failed /mn

...

Finished /mnt/h/datasets/mmCIF/yc/5yc4.cif in 0.11s
Failed /mnt/h/datasets/mmCIF/yc/5yc7.cif: Too long 287
Failed /mnt/h/datasets/mmCIF/yc/5ycg.cif: Too long 234
Finished /mnt/h/datasets/mmCIF/yc/5ycq.cif in 0.12s
Finished /mnt/h/datasets/mmCIF/yc/5ycw.cif in 0.15s
Failed /mnt/h/datasets/mmCIF/yc/6ycr.cif: Too long 255
Failed /mnt/h/datasets/mmCIF/yc/7yc6.cif: Too long 265
Failed /mnt/h/datasets/mmCIF/yc/7yce.cif: Too long 268


/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 846.
  warnings.warn(
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 848.
  warnings.warn(


Finished /mnt/h/datasets/mmCIF/yc/7ycv.cif in 0.13s
Failed /mnt/h/datasets/mmCIF/yc/7ycw.cif: Too long 276
Failed /mnt/h/datasets/mmCIF/yd/1yd0.cif: Too long 226
Failed /mnt/h/datasets/mmCIF/yd/1yd1.cif: Too long 188
Failed /mnt/h/datasets/mmCIF/yd/1yd2.cif: Too long 286
Failed /mnt/h/datasets/mmCIF/yd/1yd3.cif: Too long 269
Failed /mnt/h/datasets/mmCIF/yd/1yd4.cif: Too long 211
Failed /mnt/h/datasets/mmCIF/yd/1yd5.cif: Too long 175
Failed /mnt/h/datasets/mmCIF/yd/1yd7.cif: Too long 200
Finished /mnt/h/datasets/mmCIF/yd/1ydl.cif in 0.09s
Failed /mnt/h/datasets/mmCIF/yd/2yd2.cif: Too long 234
Failed /mnt/h/datasets/mmCIF/yd/2yd3.cif: Too long 284
Failed /mnt/h/datasets/mmCIF/yd/2yd5.cif: Too long 258
Failed /mnt/h/datasets/mmCIF/yd/2yd8.cif: Too long 295
Failed /mnt/h/datasets/mmCIF/yd/2ydb.cif: Too long 274
Failed /mnt/h/datasets/mmCIF/yd/2ydc.cif: Too long 296
Failed /mnt/h/datasets/mmCIF/yd/2ydd.cif: Too long 244
Failed /mnt/h/datasets/mmCIF/yd/2ydg.cif: Too long 206
Finished /mnt/h/

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 220.
  warnings.warn(
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 221.
  warnings.warn(
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 222.
  warnings.warn(
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 223.
  warnings.warn(
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 224.
  warnings.warn(
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/Structur

Failed /mnt/h/datasets/mmCIF/yg/2ygn.cif: Too long 232
Failed /mnt/h/datasets/mmCIF/yg/2ygp.cif: Too long 295
Failed /mnt/h/datasets/mmCIF/yg/2ygs.cif: Too long 228
Failed /mnt/h/datasets/mmCIF/yg/4yg3.cif: Too long 247
Failed /mnt/h/datasets/mmCIF/yg/4ygg.cif: Too long 273
Failed /mnt/h/datasets/mmCIF/yg/4ygh.cif: Too long 228
Failed /mnt/h/datasets/mmCIF/yg/4ygi.cif: Too long 179
Failed /mnt/h/datasets/mmCIF/yg/4ygt.cif: Too long 203
Failed /mnt/h/datasets/mmCIF/yg/4ygw.cif: Too long 166
Failed /mnt/h/datasets/mmCIF/yg/4ygz.cif: Too long 261
Failed /mnt/h/datasets/mmCIF/yg/5ygh.cif: Too long 233
Failed /mnt/h/datasets/mmCIF/yg/6yg5.cif: Too long 276
Failed /mnt/h/datasets/mmCIF/yg/7ygk.cif: Too long 207
Finished /mnt/h/datasets/mmCIF/yg/7ygv.cif in 0.13s
Failed /mnt/h/datasets/mmCIF/yg/7ygw.cif: Too long 139
Finished /mnt/h/datasets/mmCIF/yg/7ygy.cif in 0.12s
Failed /mnt/h/datasets/mmCIF/yh/1yha.cif: Too long 205
Finished /mnt/h/datasets/mmCIF/yh/1yhb.cif in 0.30s
Failed /mnt/h/datas

In [13]:
failed_paths

In [11]:
type(all_metadata)

In [12]:
print('hi')

In [24]:
# upper_direc = '/mnt/h/datasets/mmCIF/'
# upper_list = os.listdir(upper_direc)

# max_res = 3.0
# max_length = 128
# output_dir = '/mnt/h/datasets/processed_300L'


# for direc in tqdm(upper_list):
#     #unzip all files, delete original zipped files
#     fL = os.listdir(f'{upper_direc}{direc}')
#     for filegz in fL:
#         if filegz.endswith('.gz'):
#             gunzip(f'{upper_direc}{direc}/{filegz}',f'{upper_direc}{direc}/{filegz.replace(".gz","")}')
      
#     #collect mmcif files to process
#     mmcif_dir = f'{upper_direc}{direc}'
#     max_file_size = 300000000
#     min_file_size = 100
    
#     all_mmcif_paths = _retrieve_mmcif_files(mmcif_dir, max_file_size, min_file_size, False)
#     total_num_files =0
#     for mmcif_file in os.listdir(mmcif_dir):
#         if not mmcif_file.endswith('.cif'):
#             continue
#         mmcif_path = os.path.join(mmcif_dir, mmcif_file)
#         total_num_files += 1
#         if min_file_size <= os.path.getsize(mmcif_path) <= max_file_size:
#             all_mmcif_paths.append(mmcif_path)
            
            
#     all_metadata, failed_paths= process_serially2(all_mmcif_paths,max_res,300,
#                                                   output_dir, simple=False)
   
#     #delete failed paths to reduce memory usage
#     for to_del in failed_paths:
#         if os.path.isfile(to_del):
#             os.remove(to_del)

  0%|                                                                                          | 0/1060 [00:00<?, ?it/s]

Gathering mmCIF paths



  0%|                                                                                  | 1/1060 [00:00<02:00,  8.80it/s]

Processing 0 files our of 0
Finished /mnt/h/datasets/mmCIF/00/200l.cif in 0.10s
Gathering mmCIF paths



100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 730.82it/s]


...

Failed /mnt/h/datasets/mmCIF/aa/5aa0.cif: Too high resolution 5.0
Failed /mnt/h/datasets/mmCIF/aa/5aa1.cif: Too long 1807
Failed /mnt/h/datasets/mmCIF/aa/5aa2.cif: Too long 1745
Failed /mnt/h/datasets/mmCIF/aa/5aa3.cif: Too high resolution 3.2
Failed /mnt/h/datasets/mmCIF/aa/5aa4.cif: Too long 1826
Failed /mnt/h/datasets/mmCIF/aa/5aa5.cif: Too long 5462
Failed /mnt/h/datasets/mmCIF/aa/5aa6.cif: Too long 5578
Failed /mnt/h/datasets/mmCIF/aa/5aa7.cif: Too long 614
Failed /mnt/h/datasets/mmCIF/aa/5aa8.cif: Too long 464
Failed /mnt/h/datasets/mmCIF/aa/5aa9.cif: Too long 478
Failed /mnt/h/datasets/mmCIF/aa/5aaa.cif: Too long 523
Failed /mnt/h/datasets/mmCIF/aa/5aab.cif: Too long 487
Failed /mnt/h/datasets/mmCIF/aa/5aac.cif: Too long 524
Failed /mnt/h/datasets/mmCIF/aa/5aad.cif: Too high resolution 3.1
Failed /mnt/h/datasets/mmCIF/aa/5aae.cif: Too high resolution 3.11
Finished /mnt/h/datasets/mmCIF/aa/5aaf.cif in 0.20s
Finished /mnt/h/datasets/mmCIF/aa/5aag.cif in 0.36s
Failed /mnt/h/dataset

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 1085.
  warnings.warn(
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 1088.
  warnings.warn(


Failed /mnt/h/datasets/mmCIF/aa/6aag.cif: Too long 1158
Failed /mnt/h/datasets/mmCIF/aa/6aah.cif: Too long 805
Failed /mnt/h/datasets/mmCIF/aa/6aaj.cif: Too long 601
Failed /mnt/h/datasets/mmCIF/aa/6aak.cif: Too long 1181
Failed /mnt/h/datasets/mmCIF/aa/6aal.cif: Too long 569
Failed /mnt/h/datasets/mmCIF/aa/6aam.cif: Too long 315
Failed /mnt/h/datasets/mmCIF/aa/6aan.cif: Too long 498
Failed /mnt/h/datasets/mmCIF/aa/6aao.cif: Too long 566
Failed /mnt/h/datasets/mmCIF/aa/6aap.cif: Too long 545
Failed /mnt/h/datasets/mmCIF/aa/6aaq.cif: Too long 583
Failed /mnt/h/datasets/mmCIF/aa/6aar.cif: Too long 462
Failed /mnt/h/datasets/mmCIF/aa/6aas.cif: Encountered errors {('6aas', ''): 'No protein chains found in this file.'}
Failed /mnt/h/datasets/mmCIF/aa/6aau.cif: Encountered errors {('6aau', ''): 'No protein chains found in this file.'}
Failed /mnt/h/datasets/mmCIF/aa/6aav.cif: Too long 1678
Finished /mnt/h/datasets/mmCIF/aa/6aaw.cif in 0.12s


/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 844.
  warnings.warn(
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 875.
  warnings.warn(


Failed /mnt/h/datasets/mmCIF/aa/6aax.cif: Too high resolution 3.0
Failed /mnt/h/datasets/mmCIF/aa/6aay.cif: Too long 1288
Failed /mnt/h/datasets/mmCIF/aa/6aaz.cif: Too long 551
Failed /mnt/h/datasets/mmCIF/aa/7aa0.cif: Too long 425
Failed /mnt/h/datasets/mmCIF/aa/7aa1.cif: Mdtraj failed with error %c requires int or char
Failed /mnt/h/datasets/mmCIF/aa/7aa2.cif: Too long 2845
Failed /mnt/h/datasets/mmCIF/aa/7aa3.cif: Too high resolution 3.56
Failed /mnt/h/datasets/mmCIF/aa/7aa4.cif: Too long 355
Failed /mnt/h/datasets/mmCIF/aa/7aa5.cif: Too high resolution 4.18
Failed /mnt/h/datasets/mmCIF/aa/7aa7.cif: Too long 414
Finished /mnt/h/datasets/mmCIF/aa/7aa8.cif in 0.46s
Failed /mnt/h/datasets/mmCIF/aa/7aa9.cif: Too long 1501
Failed /mnt/h/datasets/mmCIF/aa/7aaa.cif: Too long 1064
Failed /mnt/h/datasets/mmCIF/aa/7aab.cif: Too long 727
Failed /mnt/h/datasets/mmCIF/aa/7aac.cif: Too long 1186
Failed /mnt/h/datasets/mmCIF/aa/7aad.cif: Too long 810
Failed /mnt/h/datasets/mmCIF/aa/7aae.cif: Too l

In [8]:
upper_direc = '/mnt/h/datasets/mmCIF/'
upper_list = os.listdir(upper_direc)

In [9]:
direc = upper_list[1]
print(direc)

01


In [10]:
direc = '/mnt/h/datasets/mmCIF/0g/'

In [11]:
fL = os.listdir(direc)
for filegz in fL:
    if filegz.endswith('.gz'):
        gunzip(f'{direc}{filegz}',f'{direc}{filegz.replace(".gz","")}')
        

In [12]:
fL = os.listdir(direc)

In [13]:
mmcif_dir = direc
max_file_size = 300000000
min_file_size = 100

In [14]:
all_mmcif_paths = _retrieve_mmcif_files(mmcif_dir, max_file_size, min_file_size, False)
total_num_files =0
for mmcif_file in os.listdir(mmcif_dir):
    if not mmcif_file.endswith('.cif'):
        continue
    mmcif_path = os.path.join(mmcif_dir, mmcif_file)
    total_num_files += 1
    if min_file_size <= os.path.getsize(mmcif_path) <= max_file_size:
        all_mmcif_paths.append(mmcif_path)

Gathering mmCIF paths


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 371.65it/s]

Processing 0 files our of 0


In [15]:
all_metadata, failed_paths= process_serially2(all_mmcif_paths,3.0,300,'/mnt/h/datasets/processed_300L',simple=False)

Failed /mnt/h/datasets/mmCIF/0g/10gs.cif: Too long 589
Failed /mnt/h/datasets/mmCIF/0g/20gs.cif: Too long 521


In [16]:
for to_del in failed_paths:
    if os.path.isfile(to_del):
        os.remove(to_del)